# Denoising EXR

In [3]:
import OpenEXR
import Imath
import numpy as np
import scipy.ndimage

def read_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    size = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    channels = {}
    for channel_name in exr_file.header()['channels'].keys():
        channel_data = exr_file.channel(channel_name, Imath.PixelType(Imath.PixelType.FLOAT))
        channel = np.frombuffer(channel_data, dtype=np.float32).reshape(size[1], size[0])
        channels[channel_name] = channel
    
    return channels, size

def write_exr(file_path, channels, size):
    header = OpenEXR.Header(size[0], size[1])
    for channel_name in channels.keys():
        header['channels'][channel_name] = Imath.Channel(Imath.PixelType(Imath.PixelType.FLOAT))
    
    exr_file = OpenEXR.OutputFile(file_path, header)
    channel_data = {name: data.tobytes() for name, data in channels.items()}
    exr_file.writePixels(channel_data)

def denoise_image(channels):
    denoised_channels = {}
    for channel_name, channel in channels.items():
        denoised_channel = scipy.ndimage.median_filter(channel, size=10)
        denoised_channels[channel_name] = denoised_channel
    return denoised_channels

def main(input_file, output_file):
    channels, size = read_exr(input_file)
    denoised_channels = denoise_image(channels)
    write_exr(output_file, denoised_channels, size)


if __name__ == "__main__":
    input_file = "src/noise.exr"
    output_file = "output/denoised_noise.exr"
    main(input_file, output_file)
